<a href="https://colab.research.google.com/github/romera-rocio/Titanic_Rocio_Romera/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estructura

* Hipótesis

* EDA básico

* Ingeniería de variables avanzada

* Preprocesamiento (numérico + categórico)

* Pipelines

* Modelos (Decision Tree, KNN, Random Forest)

* Selección del mejor modelo

* Matrices de confusión

* ROC AUC

* Predicción final para test.csv




# Hipótesis del análisis (versión H0/H1)

**H1. Sexo**
- H0: El sexo del pasajero no afecta la probabilidad de supervivencia.  
- H1: El sexo del pasajero sí afecta la probabilidad de supervivencia.

**H2. Clase (Pclass)**
- H0: La clase socioeconómica no influye en la supervivencia.  
- H1: La clase socioeconómica sí influye en la supervivencia.

**H3. Edad (Age)**
- H0: La edad del pasajero no está asociada con la supervivencia.  
- H1: La edad del pasajero sí está asociada con la supervivencia.

**H4. Tarifa (Fare)**
- H0: La tarifa pagada no afecta la probabilidad de supervivencia.  
- H1: La tarifa pagada sí afecta la probabilidad de supervivencia.

**H5. Puerto de embarque (Embarked)**
- H0: El puerto de embarque no influye en la supervivencia.  
- H1: El puerto de embarque sí influye en la supervivencia.

**H6. Tamaño familiar (FamilySize)**
- H0: El tamaño familiar no está asociado con la supervivencia.  
- H1: El tamaño familiar sí está asociado con la supervivencia.

**H7. Registro de cabina (Cabin / Deck)**
- H0: Tener registro de cabina no afecta la probabilidad de supervivencia.  
- H1: Tener registro de cabina sí afecta la probabilidad de supervivencia.

**H8. Título social (Title)**
- H0: El título social del pasajero no aporta información relevante para predecir la supervivencia.  
- H1: El título social del pasajero sí aporta información relevante.

**H9. Interacción sexo × clase**
- H0: La interacción entre sexo y clase no mejora la explicación de la supervivencia.  
- H1: La interacción entre sexo y clase sí mejora la explicación de la supervivencia.

**H10. Patrón del ticket (Ticket prefix)**
- H0: Los prefijos del ticket no están asociados a la supervivencia.  
- H1: Los prefijos del ticket sí están asociados a la supervivencia.

**H11. Interacción Fare × Pclass**
- H0: La combinación de tarifa y clase no afecta la probabilidad de supervivencia.  
- H1: La combinación de tarifa y clase sí afecta la probabilidad de supervivencia.

**H12. Modelos no lineales**
- H0: Los modelos no lineales no mejoran el rendimiento respecto a modelos lineales.  
- H1: Los modelos no lineales sí mejoran el rendimiento respecto a modelos lineales.

**H13. Estandarización**
- H0: Escalar las variables numéricas no mejora el rendimiento de los modelos sensibles a escala.  
- H1: Escalar las variables numéricas sí mejora el rendimiento de los modelos sensibles a escala.

**H14. Imputación avanzada de edad**
- H0: La imputación avanzada (RF/KNN) no mejora la predicción respecto a imputación simple.  
- H1: La imputación avanzada sí mejora la predicción.

**H15. Features derivadas**
- H0: Las variables creadas (FamilySize, IsAlone, Title, Deck) no incrementan el rendimiento de los modelos.  
- H1: Las variables creadas sí incrementan el rendimiento de los modelos.


## Importacion de Data set

In [72]:
!pip install kaggle


In [73]:
from google.colab import files
files.upload()   # elegís kaggle.json desde tu PC


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"rociobelenromerasily","key":"208666ac7f2897d87da9c755267e4620"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic
!unzip titanic.zip


titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Importacion de librerias



1.   Preprosesamiento y Graficos : Pandas , Numpy, Matplotlib y Seaborn

2.   PostProcesamiento: Scikit-learn( algoritmos de aprendizaje automático ) y SciPy ( Test estadisticos )





In [ ]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# Preprocessing & ML
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.impute import SimpleImputer

# ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Estadística
from scipy.stats import zscore, iqr, shapiro, ks_2samp, mannwhitneyu, chi2_contingency

import warnings
warnings.filterwarnings("ignore")


## Exploracion de Data Set

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
gender_submission = pd.read_csv("gender_submission.csv")

train.head()


# Analisis Estadistico Descriptivo

## Conclusiones Descriptivas

- **Sexo**: La tasa de supervivencia femenina (74%) es marcadamente superior a la masculina (19%), mostrando un fuerte efecto del género.
- **Clase**: Las clases 1 y 2 presentan mayor supervivencia; la clase 3 concentra la mayoría de fallecidos.
- **Edad**: La distribución es asimétrica; predomina la franja 20–30 años. No sugiere normalidad.


In [ ]:
print("Información general del dataset:")
train.info()

print("\nValores faltantes por columna:")
print(train.isna().sum())

train.describe()


##GRAFICOS


In [ ]:
# === Cell 8 (Code) ===
# EDA Bivariado: relaciones clave
fig, axes = plt.subplots(2,2, figsize=(14,10))

sns.countplot(data=train, x="Sex", hue="Survived", ax=axes[0,0])
axes[0,0].set_title("Survival by Sex")

sns.countplot(data=train, x="Pclass", hue="Survived", ax=axes[0,1])
axes[0,1].set_title("Survival by Pclass")

sns.kdeplot(data=train, x="Age", hue="Survived", ax=axes[1,0])
axes[1,0].set_title("Age distribution by Survived")

sns.boxplot(data=train, x="Survived", y="Fare", ax=axes[1,1])
axes[1,1].set_title("Fare vs Survived")

plt.tight_layout()
plt.show()


## Conclusión técnica sobre valores faltantes

- Existe un patrón de *missingness no uniforme*.  
- `Cabin` (77.10% NA) debe ser excluida como variable informativa directa debido a su escasa completitud; un indicador binario preserva utilidad estructural.
- `Age` (19.8% NA) requiere **imputación supervisada** para evitar sesgo en subpoblaciones jóvenes/adultas.
- `Embarked` (0.22% NA) admite imputación por **moda** sin afectar la varianza residual.
- No se detecta un patrón MCAR global; las ausencias se distribuyen con dependencia en Pclass y Sex.


In [ ]:
# === Cell 6 (Code) ===
# Heatmap de missing values y resumen rápido
plt.figure(figsize=(10,4))
sns.heatmap(train.isna(), cbar=False)
plt.title("Mapa de valores faltantes (train)")
plt.show()

# Imprimir columnas con mayor missing
dqr.head(10)


In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,4))

sns.countplot(data=train, x='Sex', hue='Survived', ax=axes[0])
axes[0].set_title('Survival by Sex')

sns.countplot(data=train, x='Pclass', hue='Survived', ax=axes[1])
axes[1].set_title('Survival by Pclass')

plt.figure(figsize=(6,4))
sns.histplot(train['Age'].dropna(), kde=True)
plt.title('Age distribution')
plt.show()


#TESTS ESTADÍSTICOS Y OUTLIERS

#Outliers (Z-score + IQR)

## Conclusión técnica sobre outliers

- **Age** presenta pocos valores extremos → no requiere winsorización ni recorte.
- **Fare** exhibe numerosos outliers y fuerte asimetría → se recomienda aplicar **log-transform** para estabilizar la varianza.
- La dispersión en `Fare` es informativa (mayor tarifa → mayor supervivencia), por lo que NO deben eliminarse outliers.


In [ ]:
num_cols = ["Age", "Fare"]

for col in num_cols:
    print("\n========================")
    print(f"OUTLIERS EN: {col}")
    print("========================")

    z = np.abs(zscore(train[col].dropna()))
    out_z = (z > 3).sum()
    print(f"Z-score > 3: {out_z} outliers")

    Q1 = train[col].quantile(0.25)
    Q3 = train[col].quantile(0.75)
    I = Q3 - Q1
    lower = Q1 - 1.5*I
    upper = Q3 + 1.5*I
    out_iqr = ((train[col] < lower) | (train[col] > upper)).sum()
    print(f"IQR Rule: {out_iqr} outliers")

    print(f"Límites sugeridos → [{lower:.2f}, {upper:.2f}]")


##Normalidad (Shapiro + KS)

## Evaluación de Normalidad y Diferencias entre Grupos

### Age

* **Shapiro–Wilk:** p < 0.001 → rechazo de la hipótesis de normalidad.
* **Kolmogorov–Smirnov (Survived vs Dead):** p = 0.123 → no se detecta diferencia significativa en la distribución de Age entre grupos.
* **Interpretación:** Age no sigue distribución normal; su efecto aislado sobre supervivencia es débil.

### Fare

* **Shapiro–Wilk:** p < 0.001 → distribución claramente no normal.
* **Kolmogorov–Smirnov (Survived vs Dead):** p < 0.001 → diferencia significativa entre grupos.
* **Interpretación:** Fare exhibe sesgo positivo y diferencia estadísticamente significativa según supervivencia; se recomienda **log-transform** para estabilizar varianza antes de modelado.


In [ ]:
for col in ["Age","Fare"]:
    print("\n========================")
    print(f"TESTS DE NORMALIDAD: {col}")
    print("========================")

    w, p = shapiro(train[col].dropna())
    print(f"Shapiro–Wilk p = {p:.5f}")

    a = train.loc[train["Survived"]==1, col].dropna()
    b = train.loc[train["Survived"]==0, col].dropna()
    ks_stat, ks_p = ks_2samp(a,b)
    print(f"KS test p = {ks_p:.5f}")


# 3. Correlaciones :Pearson, Spearman y Cramér’s V

## Conclusiones del EDA avanzado

- **Sex vs Survived — Chi² p < 0.00001** → existe dependencia estadísticamente significativa.
- **Pclass vs Survived — Chi² p < 0.00001** → la clase impacta fuertemente en supervivencia.
- **Age — Mann-Whitney p = alta (no significativa)** → las distribuciones de edad son similares entre grupos.
- `Fare` muestra valores sistemáticamente mayores en sobrevivientes → variable útil para el modelo.
- La combinación **(Sex + Pclass + Fare)** representa la estructura predictiva más fuerte del dataset.


In [ ]:
# === Cell 11 (Code) ===
# Correlaciones: Pearson (num), Spearman (ordinal), Cramér's V (cat)

import scipy.stats as ss

def cramers_v(x, y):
    """Cramér's V para dos variables categóricas."""
    cm = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(cm)[0]
    n = cm.sum().sum()
    r, k = cm.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))


# ==============================
# Cálculo de métricas
# ==============================

# Pearson (numérico vs binario)
pearson_age, _   = pearsonr(train["Age"].fillna(train["Age"].median()), train["Survived"])
pearson_fare, _  = pearsonr(train["Fare"].fillna(train["Fare"].median()), train["Survived"])

# Spearman (ordinal vs binario)
spearman_pclass  = spearmanr(train["Pclass"], train["Survived"]).correlation

# Cramér's V (categorical vs binario)
cramers_sex      = cramers_v(train["Sex"], train["Survived"])
cramers_embarked = cramers_v(train["Embarked"].fillna("Missing"), train["Survived"])

# ==============================
# Tabla resumen
# ==============================

metrics = pd.DataFrame({
    "feature": ["Age", "Fare", "Pclass", "Sex", "Embarked"],
    "metric":  ["Pearson r", "Pearson r", "Spearman ρ", "Cramér's V", "Cramér's V"],
    "value":   [pearson_age, pearson_fare, spearman_pclass, cramers_sex, cramers_embarked]
})

display(metrics)


# ==============================
# Prints técnicos
# ==============================

print("\n=== Pearson Correlation ===")
print(f"Age:  r={pearson_age:.3f}")
print(f"Fare: r={pearson_fare:.3f}")

print("\n=== Spearman (Ordinal) ===")
print(f"Pclass: ρ={spearman_pclass:.3f}")

print("\n=== Cramér's V (Categorical) ===")
print(f"Sex:      V={cramers_sex:.3f}")
print(f"Embarked: V={cramers_embarked:.3f}")


In [ ]:
ct_sex = pd.crosstab(train['Sex'], train['Survived'])
chi2_sex, p_sex, _, _ = chi2_contingency(ct_sex)
print("Sex vs Survived — chi2: %.3f p: %.5f" % (chi2_sex, p_sex))

ct_pclass = pd.crosstab(train['Pclass'], train['Survived'])
chi2_pc, p_pc, _, _ = chi2_contingency(ct_pclass)
print("Pclass vs Survived — chi2: %.3f p: %.5f" % (chi2_pc, p_pc))

age_survived = train.loc[train['Survived']==1,'Age'].dropna()
age_dead = train.loc[train['Survived']==0,'Age'].dropna()
stat_age,p_age = mannwhitneyu(age_survived, age_dead)
print("Mann-Whitney Age p=",p_age)

fig,axes = plt.subplots(2,2, figsize=(12,8))
sns.countplot(data=train,x='Sex',hue='Survived',ax=axes[0,0])
sns.countplot(data=train,x='Pclass',hue='Survived',ax=axes[0,1])
sns.kdeplot(data=train,x='Age',hue='Survived',ax=axes[1,0])
sns.boxplot(data=train,x='Survived',y='Fare',ax=axes[1,1])
plt.tight_layout()
plt.show()
